## Test in real time

In [3]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

from tensorflow import keras

In [4]:
# mp holistic for keypoints
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities


# functions for drawing landmarks
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results


# drawing landmarks
def draw_styled_landmarks(image, results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

    
# preparing vector of date
from numpy import zeros
def extract_keypoints(results):
    # for pose - FACE INCLUDED
    if (results.pose_landmarks == None):
        face = []
        pose_left = []
        pose_right = []
    else:
        x = 0
        y = 0
        #z = 0
        for i in range(11):
            if results.pose_landmarks.landmark[i].visibility > 0.95:
                x += results.pose_landmarks.landmark[i].x
                y += results.pose_landmarks.landmark[i].y
                #z += results.pose_landmarks.landmark[i].z
            else:
                return [], [], [], [], [], # no mercy jak nie ma jakiegos puntku na twarzy to jest nasz punkt odniesienia
        x = x/11
        y = y/11
        #z = z/11
        face = np.array([x,y])
        #face = np.array([x,y,z])
        
        #meme = []
        pose_left = []
        pose_right = []
        
        # middle point in chest
        if(results.pose_landmarks.landmark[11].visibility > 0.85 and results.pose_landmarks.landmark[12].visibility > 0.85):
            pose_left.append(((results.pose_landmarks.landmark[11].x + results.pose_landmarks.landmark[12].x) /2) - face[0])
            pose_left.append(((results.pose_landmarks.landmark[11].y + results.pose_landmarks.landmark[12].y) /2) - face[1])
            pose_right.append(((results.pose_landmarks.landmark[11].x + results.pose_landmarks.landmark[12].x) /2) - face[0])
            pose_right.append(((results.pose_landmarks.landmark[11].y + results.pose_landmarks.landmark[12].y) /2) - face[1])
        else:
            pose_left.append(0)
            pose_left.append(0)
            pose_right.append(0)
            pose_right.append(0)

        for i in range(11, 17):
            if(results.pose_landmarks.landmark[i].visibility > 0.85): # to jeszcze do empirycznego ustawienia !!!!!!!!
                if(i % 2 == 0):
                    pose_right.append(results.pose_landmarks.landmark[i].x - face[0])
                    pose_right.append(results.pose_landmarks.landmark[i].y - face[1])
                else:     
                    pose_left.append(results.pose_landmarks.landmark[i].x - face[0])
                    pose_left.append(results.pose_landmarks.landmark[i].y - face[1])
                
                #meme.append(results.pose_landmarks.landmark[i].visibility)
            else:
                if(i % 2 == 0):
                    pose_right.append(0)
                    pose_right.append(0) 
                else:
                    pose_left.append(0)
                    pose_left.append(0)     
                                      
                #meme.append(-1 * results.pose_landmarks.landmark[i].visibility)

        pose_left = np.array(pose_left)
        pose_right = np.array(pose_right)
        #vis = np.array(meme)

    # for right hand
    if (results.right_hand_landmarks == None):
        right_hand = []
    else:
        right_hand = np.array([[-1 * (res.x - face[0]), res.y - face[1]] for res in results.right_hand_landmarks.landmark]).flatten()

    # for left hand
    if (results.left_hand_landmarks == None):
        left_hand = []
    else:
        left_hand = np.array([[res.x - face[0], res.y - face[1]] for res in results.left_hand_landmarks.landmark]).flatten()
    
    return face, pose_left, pose_right, left_hand, right_hand


#Viz logic
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

# folders info
no_sequences = 50 # 50 videos
sequence_length = 12 # for 20 frames   
DATA_PATH = os.path.join('MP_Data') 

In [7]:
model = keras.models.load_model("action2d_e25 _96.h5")
#actions = np.array(['hello','eat','I_me','hurt','need','thanks','sound_of_sand_on_deserts','drink']) 
actions = np.array(['', 'hello','I_me','need','thanks', 'drink', 'beer'])
colors = [(255, 153, 51) ,(245,117,16), (117,245,16), (16,117,245),(245,117,16), (117,245,16), (16,117,245)]

In [10]:
# 1. New detection variables
all_keypoints = [] # stare sequance
frame_num = 1
sentence = []
threshold = 0.9

blank_pose_left = 0
all_blank_pose_left = []
all_frames_pose_left = []

blank_pose_right = 0
all_blank_pose_right = []
all_frames_pose_right = []

blank_left_hand = 0
all_blank_left_hand = []
all_frames_left_hand = []

blank_right_hand = 0
all_blank_right_hand = []
all_frames_right_hand = []

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.4, min_tracking_confidence=0.4) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
        
        ## Making contrast i can put later to mediapipe detection to not have contrast all the time on video
        lab= cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
        l_channel, a, b = cv2.split(lab)
        # Applying CLAHE to L-channel
        # feel free to try different values for the limit and grid size:
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        cl = clahe.apply(l_channel)
        # merge the CLAHE enhanced L-channel with the a and b channel
        limg = cv2.merge((cl,a,b))
        # Converting image from LAB Color model to BGR color spcae
        frame = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        # print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # extracting keypoints
        face, pose_left, pose_right, left_hand, right_hand = extract_keypoints(results)
        if len(face) == 0:
            print("nic nie ma na ekraie")
            # liczymy od nowa :<
            frame_num = 1
                
            blank_pose_left = 0
            all_blank_pose_left = []
            all_frames_pose_left = []

            blank_pose_right = 0
            all_blank_pose_right = []
            all_frames_pose_right = []

            blank_left_hand = 0
            all_blank_left_hand = []
            all_frames_left_hand = []

            blank_right_hand = 0
            all_blank_right_hand = []
            all_frames_right_hand = []
        else:
            # prawe i lewe strony ciala tez trzeba przebadac
            if len(left_hand) == 0:
                left_hand = np.zeros(42)
                all_blank_left_hand.append(1) #2
                #print("nie ma lewej dloni na ekranie")
                # zliczamy
            else:
                all_blank_left_hand.append(0)
            if len(right_hand) == 0:
                right_hand = np.zeros(42)
                all_blank_right_hand.append(1)
                #print("nie ma prawej dloni na ekranie")
                #zliaczamy
            else:
                all_blank_right_hand.append(0)
            if len(pose_left) - np.count_nonzero(pose_left) != 0:
                all_blank_pose_left.append(1)
                #print("nie ma lewego ramienia")
                #zliczamy
            else:
                all_blank_pose_left.append(0)
            if len(pose_right) - np.count_nonzero(pose_right) != 0:
                all_blank_pose_right.append(1)
                #print("nie ma prawego ramienia")
                #zliczamy
            else:
                all_blank_pose_right.append(0)
                
            # ilosc dziur na 12 klatkach
            all_blank_pose_left = all_blank_pose_left[-12:]
            all_blank_pose_right = all_blank_pose_right[-12:]
            all_blank_left_hand = all_blank_left_hand[-12:]
            all_blank_right_hand = all_blank_right_hand[-12:]
                
            blank_pose_left = np.count_nonzero(all_blank_pose_left)
            blank_pose_right = np.count_nonzero(all_blank_pose_right)
            blank_left_hand = np.count_nonzero(all_blank_left_hand)
            blank_right_hand = np.count_nonzero(all_blank_right_hand)
                                
            # pierwsza klatka musi miec wartosci chociaz jedna reke i ramie
            if (frame_num == 1 and (blank_left_hand + blank_pose_left == 0 or blank_right_hand + blank_pose_right == 0)):
                # cala historia robienia znaku
                all_frames_pose_left.append(pose_left)
                all_frames_pose_right.append(pose_right)
                all_frames_left_hand.append(left_hand)
                all_frames_right_hand.append(right_hand)
                frame_num += 1
            elif (frame_num != 1):
                # cala historia robienia znaku
                all_frames_pose_left.append(pose_left)
                all_frames_pose_right.append(pose_right)
                all_frames_left_hand.append(left_hand)
                all_frames_right_hand.append(right_hand)
                    
                if (frame_num != sequence_length):
                    frame_num += 1 
                else:
                    # cala historia robienia znaku                        
                    all_frames_pose_left = all_frames_pose_left[-12:]
                    all_frames_pose_right = all_frames_pose_right[-12:]
                    all_frames_left_hand = all_frames_left_hand[-12:]
                    all_frames_right_hand = all_frames_right_hand[-12:]
                    
                    if (blank_left_hand > 2 or blank_pose_left > 2) and (blank_right_hand > 2 or blank_pose_right > 2):
                        print("za malo landmarkow")                  
                    else:    
                        #print("blank_pose_left:", blank_pose_left, " blank_left_hand:",  blank_left_hand, "  blank_pose_right:", blank_pose_right,  " blank_right_hand:", blank_right_hand)                
                        s1 = False
                        s2 = False
                        # czy jest z czego wyznaczac czy nie ma za duzo dziur dla lewej reki
                        if blank_left_hand <= 2 and blank_pose_left <= 2:
                            varations = []
                            col = []
                            for it in range(42):
                                for frame in all_frames_left_hand:
                                    col.append(10*frame[it])   # 1 dobrze wyznacza
                                # eliminacja zer w wektorze dla dloni
                                for it2 in range(1, len(col)):
                                    if it2 == len(col)-1:
                                        if(col[it2] == 0):
                                            col[it2] = col[it2-1]
                                            all_frames_left_hand[it2][it] = col[it2-1]
                                    else:
                                        if(col[it2] == 0):
                                            if(col[it2+1] == 0):
                                                col[it2] = col[it2-1]
                                                all_frames_left_hand[it2][it] = col[it2-1]
                                            else:
                                                col[it2] = (col[it2-1] + col[it2+1])/2
                                                all_frames_left_hand[it2][it] = (col[it2-1] + col[it2+1])/2  

                                # wyliczamy wariancje
                                srednio =  sum(col)/sequence_length
                                varations.append(np.sum( (col - srednio)**2 ) / len(col))
                                col = []
                            # decycja na podstawie wariancji 
                            if(np.sum(varations) >= 0.8):
                                #print("suma: ", np.sum(varations), "lewa dlon giboecze")
                                s1 = True
                            #else:
                                #print("suma: ", np.sum(varations), "lewa dlon sie nie poruszala")

                            # eliminacja zer w wektorze dla pose
                            col = []
                            for it in range(8):
                                for frame in all_frames_pose_left:
                                    col.append(frame[it])   # 1 dobrze wyznacza

                                for it2 in range(1, len(col)):
                                    if it2 == len(col)-1:
                                        if(col[it2] == 0):
                                            col[it2] = col[it2-1]
                                            all_frames_pose_left[it2][it] = col[it2-1]
                                    else:
                                        if(col[it2] == 0):
                                            if(col[it2+1] == 0):
                                                col[it2] = col[it2-1]
                                                all_frames_pose_left[it2][it] = col[it2-1]
                                            else:
                                                col[it2] = (col[it2-1] + col[it2+1])/2
                                                all_frames_pose_left[it2][it] = (col[it2-1] + col[it2+1])/2
                                col = []
                        #else:
                            #print("lewa reka jest podziurawiona albo nie ma jej w kadrze")


                        # czy jest z czego wyznaczac czy nie ma za duzo dziur dla prawej reki
                        if blank_right_hand <= 2 and blank_pose_right <= 2:
                            varations = []
                            col = []
                            for it in range(42):
                                for frame in all_frames_right_hand:
                                    col.append(10*frame[it])   # 1 dobrze wyznacza
                                # eliminacja zer w wektorze dla dloni
                                for it2 in range(1, len(col)):
                                    if it2 == len(col)-1:
                                        if(col[it2] == 0):
                                            col[it2] = col[it2-1]
                                            all_frames_right_hand[it2][it] = col[it2-1]
                                    else:
                                        if(col[it2] == 0):
                                            if(col[it2+1] == 0):
                                                col[it2] = col[it2-1]
                                                all_frames_right_hand[it2][it] = col[it2-1]
                                            else:
                                                col[it2] = (col[it2-1] + col[it2+1])/2
                                                all_frames_right_hand[it2][it] = (col[it2-1] + col[it2+1])/2

                                # wyliczamy wariancje   
                                srednio =  sum(col)/sequence_length
                                varations.append(np.sum( (col - srednio)**2 ) / len(col))
                                col = []
                            # decycja na podstawie wariancji    
                            if(np.sum(varations) >= 0.8):
                                #print("suma: ", np.sum(varations), "prawa dlon giboecze")
                                s2 = True
                            #else:
                                #print("suma: ", np.sum(varations), "prawa dlon sie nie poruszala")

                            # # eliminacja zer w wektorze dla pose
                            col = []
                            for it in range(8):
                                for frame in all_frames_pose_right:
                                    col.append(frame[it])   # 1 dobrze wyznacza

                                for it2 in range(1, len(col)):
                                    if it2 == len(col)-1:
                                        if(col[it2] == 0):
                                            col[it2] = col[it2-1]
                                            all_frames_pose_right[it2][it] = col[it2-1]
                                    else:
                                        if(col[it2] == 0):
                                            if(col[it2+1] == 0):
                                                col[it2] = col[it2-1]
                                                all_frames_pose_right[it2][it] = col[it2-1]
                                            else:
                                                col[it2] = (col[it2-1] + col[it2+1])/2
                                                all_frames_pose_right[it2][it] = (col[it2-1] + col[it2+1])/2
                                col = []
                        #else:
                            #print("prawa reka jest podziurawiona albo nie ma jej w kadrze")


                        # Decyzja czy to jedno czy 2 reczny znak
                        if (s1 and s2):
                            print("do sieci na 2 rece")
                            for i in range(sequence_length):
                                keypoints = np.concatenate([all_frames_pose_left[i], all_frames_left_hand[i], all_frames_pose_right[i], all_frames_right_hand[i]])                                
                                all_keypoints.append(keypoints)
                                
                            #------ teraz podobnie jak bylo w pierwowzorze ----
                            # To do: siec na 2 lapy
                            # ------ ------- ------ ------ ------ ------ ------                               
                            all_keypoints = [] # mozna to kiedys oszczedniej zrobic
                            print()

                        elif(s1 or s2):
                            print("do sieci dla 1 reki")
                            if s1:
                                for i in range(sequence_length):                  
                                    keypoints = np.concatenate([all_frames_pose_left[i], all_frames_left_hand[i]])
                                    all_keypoints.append(keypoints)
                            else:
                                for i in range(sequence_length):
                                    keypoints = np.concatenate([all_frames_pose_right[i], all_frames_right_hand[i]])
                                    all_keypoints.append(keypoints)
                             
                            #------ teraz podobnie jak bylo w pierwowzorze ----
                            res = model.predict(np.expand_dims(all_keypoints, axis=0))[0]
                                
                            #3. Viz logic
                            if res[np.argmax(res)] > threshold: 
                                if len(sentence) > 0:
                                    if actions[np.argmax(res)] != sentence[-1]:
                                        sentence.append(actions[np.argmax(res)])
                                else:
                                    sentence.append(actions[np.argmax(res)])

                            if len(sentence) > 5: 
                                sentence = sentence[-5:]

                            # PRZESKOCZYLEM FUNKCJA JEST NA GORZE
                            # Viz probabilities
                            image = prob_viz(res, actions, image, colors)
                            # ------ ------- ------ ------ ------ ------ ------  
                            
                            all_keypoints = [] # mozna to kiedys oszczedniej zrobic
                            print()
                        
                        else: #ten else (patrz opis ponizej w princie) - w przypadku real life nic nie wypisywac az data nie bedzie sie zgadzac
                            print("rece byly ale sie nie ruszaly albo jedna reka stala a druga zapierdalala tak ze nie zebralo dobrze landmakow")
                            print()     
            else:
                print("nie zebrano landmarkow z pierwszej klatki")
                
                blank_pose_left = 0
                all_blank_pose_left = []

                blank_pose_right = 0
                all_blank_pose_right = []

                blank_left_hand = 0
                all_blank_left_hand = []

                blank_right_hand = 0
                all_blank_right_hand = []        
            
        ## stare ##
        '''
        # zuuuu predict logic 
        keypoints = extract_keypoints(results)
        #sequence.insert(0,keypoints)
        #sequence = sequence[:30]  #1
        sequence.append(keypoints) #2
        sequence = sequence[-12:]
        
        if len(sequence) == 12:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            #print(actions[np.argmax(res)])

        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]
            
            # PRZESKOCZYLEM FUNKCJA JEST NA GORZE
            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
        '''
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        #tmp = image.copy()
        cv2.imshow('OpenCV ladnmark_test', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

nie zebrano landmarkow z pierwszej klatki
nie zebrano landmarkow z pierwszej klatki
nie zebrano landmarkow z pierwszej klatki
nie zebrano landmarkow z pierwszej klatki
nie zebrano landmarkow z pierwszej klatki
nie zebrano landmarkow z pierwszej klatki
nie zebrano landmarkow z pierwszej klatki
nie zebrano landmarkow z pierwszej klatki
nie zebrano landmarkow z pierwszej klatki
nie zebrano landmarkow z pierwszej klatki
nie zebrano landmarkow z pierwszej klatki
nie zebrano landmarkow z pierwszej klatki
nie zebrano landmarkow z pierwszej klatki
nie zebrano landmarkow z pierwszej klatki
nie zebrano landmarkow z pierwszej klatki
nie zebrano landmarkow z pierwszej klatki
nie zebrano landmarkow z pierwszej klatki
nie zebrano landmarkow z pierwszej klatki
nie zebrano landmarkow z pierwszej klatki
nie zebrano landmarkow z pierwszej klatki
nie zebrano landmarkow z pierwszej klatki
nie zebrano landmarkow z pierwszej klatki
nie zebrano landmarkow z pierwszej klatki
nie zebrano landmarkow z pierwszej

rece byly ale sie nie ruszaly albo jedna reka stala a druga zapierdalala tak ze nie zebralo dobrze landmakow

rece byly ale sie nie ruszaly albo jedna reka stala a druga zapierdalala tak ze nie zebralo dobrze landmakow

do sieci dla 1 reki
1/1 [==============================] - 0s 38ms/step

do sieci dla 1 reki
1/1 [==============================] - 0s 36ms/step

do sieci dla 1 reki
1/1 [==============================] - 0s 35ms/step

do sieci dla 1 reki
1/1 [==============================] - 0s 39ms/step

do sieci dla 1 reki
1/1 [==============================] - 0s 38ms/step

do sieci dla 1 reki
1/1 [==============================] - 0s 39ms/step

do sieci dla 1 reki
1/1 [==============================] - 0s 39ms/step

do sieci dla 1 reki
1/1 [==============================] - 0s 38ms/step

do sieci dla 1 reki
1/1 [==============================] - 0s 38ms/step

do sieci dla 1 reki
1/1 [==============================] - 0s 37ms/step

do sieci dla 1 reki
1/1 [=========================

In [30]:
# siec na 2 lapy
# zrobic dataset na 2 lapy
# zrobic 3 blanki z rzedu to od nowa* zeby zer nie ciagnelo z poczatku jak jakas reka sie pojawi z ukrycia  
# i nie byla wczesniej naprawiana to rozpierdala wariancje jak chcemy sie czegos pozbyc

NameError: name 'all_keypoitns' is not defined

## training on PC

In [3]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [11]:
actions = np.array(['', 'hello','I_me','need','thanks', 'drink', 'beer'])

In [6]:
label_map = {label:num for num, label in enumerate(actions)}

sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence+1), "{}.npy".format(frame_num+1)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
        
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [35]:
#Crossval

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(12,50)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
#model.add(LSTM(128, return_sequences=True, activation='relu'))
#model.add(LSTM(64, return_sequences=True, activation='relu'))
#model.add(LSTM(128, return_sequences=True ,activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
#model.add(Dense(32, activation='relu'))
#model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(X_train, y_train, epochs=25, callbacks=[tb_callback])
model.summary()

Epoch 1/25
11/11 [==============================] - 4s 33ms/step - loss: 1.9257 - categorical_accuracy: 0.4096
Epoch 2/25
11/11 [==============================] - 0s 29ms/step - loss: 1.7234 - categorical_accuracy: 0.6566
Epoch 3/25
11/11 [==============================] - 0s 29ms/step - loss: 1.6273 - categorical_accuracy: 0.7169
Epoch 4/25
11/11 [==============================] - 0s 30ms/step - loss: 1.0340 - categorical_accuracy: 0.6717
Epoch 5/25
11/11 [==============================] - 0s 29ms/step - loss: 0.6549 - categorical_accuracy: 0.7741
Epoch 6/25
11/11 [==============================] - 0s 29ms/step - loss: 0.5960 - categorical_accuracy: 0.7952
Epoch 7/25
11/11 [==============================] - 0s 28ms/step - loss: 0.8704 - categorical_accuracy: 0.7500
Epoch 8/25
11/11 [==============================] - 0s 29ms/step - loss: 0.7233 - categorical_accuracy: 0.7440
Epoch 9/25
11/11 [==============================] - 0s 29ms/step - loss: 0.6545 - categorical_accuracy: 0.8253
E

In [36]:
model.save('action2d_e25 _96.h5')